In [1]:
import pandas as pd
import os
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from collections import Counter
from string import punctuation
import math
import numpy as np

import pickle

### Query transformer

In [2]:
# Function to transform a given query into a list of tokenized+stemmed words (same pre-processing steps applied as it 
# was done for creating the inverted index)

def query_prep(query):
    """convert a given query into a list of tokenized and stemmed words"""
    
    stop_words = set(stopwords.words("english"))
    stemmer = PorterStemmer()
    
    word_tokens = nltk.word_tokenize(query)
    tokens_swremoved = [w for w in word_tokens if w.lower() not in stop_words]
    tokens_stemmed = [stemmer.stem(w) for w in tokens_swremoved]
    tokens_puncremoved = [token for token in tokens_stemmed if token not in punctuation]
    
    return tokens_puncremoved

In [3]:
# load inverted index
with open("app/inv_index.pickle", "rb") as file:
    inv_index = pickle.load(file)

### Retrieval function with TF maximum frequency normalization

In [4]:
# function to retrieve and rank the webpages/documents given a query

def retrieve_n_rank_docs(inverted_index, queries, max_docs=-1):
    """
    Retrieve webpages in order of relevance from an inverted index based on a query. The function looks only 
    at terms of the query which are present in the inverted index. The ranking is based on the retrieval function 
    S(D, Q)=sum(TF(w,D)*IDF(w)*QTF(w)), where TF is caclulated using maximum frequency 
    normalization TF(w,D)=0.5+(0.5xc(w,D)/MaxFreq_w(D)), and IDF(w)=1+ln(n/k). The query term frequency (QTF) is just
    the occurrence of the word in the query. 
    
    - input: inverted index, query, max_docs (max number of docs to be retrieved)
    - returns the webpages in descending order of relevance
    
    """
      
    # dict for collecting maxFreqw(D) for calulation of normlized TF using maximum frequency normalization, 
    # traverse through all documents in the given inverted index and search for maximum frequency of a word in a document
    max_freq_wD_dict = {}
    for token, posting in inverted_index.items():
        for key, val in posting.items():
            if key not in max_freq_wD_dict:
                max_freq_wD_dict[key] = val[0]
            elif val[0] > max_freq_wD_dict[key]:
                max_freq_wD_dict[key] = val[0]
                
    ret_docs= {}

    for query in queries:
        # dict for each query (temporary)
        docs = {}
        for token in queries[query]:
            # check if token is in index
            if token in inverted_index.keys():
                # IDF: formula: 1 + ln(N/df(w)), Note: IDF is not dependent on a particular document!
                idf = 1 + math.log(len(max_freq_wD_dict) / len(inverted_index[token]))
                # traverse through the documents in the inverted index for the given token, calculate TF*IDF and
                # store the per-document accumulated version of it in dict docs
                # (dictionary accumulating pattern)
                for doc in inverted_index[token]:
                    # calculation of TF*IDF (with normalized version of TF using maxFreqw(D) from dict max_freq_wD_dict)
                    c_wD = inverted_index[token][doc][0]
                    max_freq_wD = max_freq_wD_dict[doc]
                    tfidf = (0.5 + (0.5 * c_wD / max_freq_wD)) * idf 
                    if doc not in docs:
                        docs[doc] = tfidf
                    else:
                        docs[doc] += tfidf
                        
        # rounding before sorting
        docs = {k: round(v,3) for k, v in docs.items()}

        # options for max_docs
        if max_docs == -1:
            docs_per_query = sorted(Counter(docs).most_common(), key=lambda x: (-x[1], int(x[0][0].split('d')[1])))
        else:
            docs_per_query = sorted(Counter(docs).most_common(), key=lambda x: (-x[1], int(x[0][0].split('d')[1])))[:max_docs]
        # grab the doc id out of docs_per_query
        ret_docs[query] = [(doc,freq) for doc, freq in docs_per_query]
                
    return ret_docs

### Okapi/BM25 Retrieval function (Robertson et al., 99)

In [5]:
# OkapiBM25 version of the retrieval function
def OkapiBM25(inverted_index, queries, max_docs=-1, k1=1.2, b=0.75, k3=1000):
    """
    Retrieve webpages in order of relevance from an inverted index based on a query. The function looks only 
    at terms of the query which are present in the inverted index. The ranking is based on the retrieval function 
    S(D, Q)=sum(((k1+1)*c_wD)/(k1*(1-b+b*(|D|/avg_dl))+c_wD) * ln((N-df(w)+0.5)/(df(w)+0.5) * ((k3+1)*c_wQ)/(k3+c_wQ)). 
    
    - input: inverted index, query, max_docs (max number of docs to be retrieved), and parameters k1, b, k3
    - returns the webpages in descending order of relevance
    
    """
    
    # dict for storing the document length
    doc_dict = {}
    for token, posting in inverted_index.items():
        for doc, val in posting.items():
            if doc not in doc_dict:
                doc_dict[doc] = val[0]
            else:
                doc_dict[doc] += val[0]
    # average doc length
    avg_dl = np.mean([value for key, value in doc_dict.items()]) 

    ret_docs= {}            

    for query in queries:
        # dict for each query (temporary)
        docs = {}
        for token in queries[query]:
            # calculate query term frequency c_wQ
            c_wQ = queries[query].count(token)
            # check if token is in index
            if token in inverted_index.keys():
                # IDF: formula: ln((N-df(w)+0.5)/(df(w)+0.5), Note: IDF is not dependent on a particular document!
                idf = math.log((len(doc_dict)-len(inverted_index[token])+0.5)/(len(inverted_index[token])+0.5))
                # traverse through the documents in the inverted index for the given token, calculate the score per token 
                # using Okapi/BM25 and store the per-document accumulated version of it in dict docs
                # (dictionary accumulating pattern)
                for doc in inverted_index[token]:
                    # TF=((k1+1)*c_wD)/(k1*(1-b+b*(|D|/avg_dl)+c_wD))
                    c_wD = inverted_index[token][doc][0]
                    tf = ((k1 + 1) * c_wD) / (k1*(1 - b + b * (doc_dict[doc]/avg_dl)) + c_wD)
                    # normalized query term frequency: qtf=((k3+1)*c_wQ)/(k3+c_wQ)
                    qtf = ((k3 + 1) * c_wQ) / (k3 + c_wQ)
                    score = tf * idf * qtf
                    if doc not in docs:
                        docs[doc] = score
                    else:
                        docs[doc] += score
                        
        # rounding before sorting
        docs = {k: round(v,3) for k, v in docs.items()}

        # options for max_docs
        if max_docs == -1:
            docs_per_query = sorted(Counter(docs).most_common(), key=lambda x: (-x[1], int(x[0][0].split('d')[1])))
        else:
            docs_per_query = sorted(Counter(docs).most_common(), key=lambda x: (-x[1], int(x[0][0].split('d')[1])))[:max_docs]
        # grab the doc id out of docs_per_query
        ret_docs[query] = [(doc,freq) for doc, freq in docs_per_query]
                
    return ret_docs

### Queries by Users Trevor, Julien, André

In [6]:
# queries from users
query_list = ['speech language pathology',
              'mental health in college',
              'healthcare in different countries',
              'major health legislation',
              'impact of climate change',
              'dental healthcare united states',
              'help my child develop healthy behaviors',
              'relationship between womens rights and health outcomes',
              'healthcare in asian countries',
              'effects of smoking',
              'LGBTQ healthcare',
              'country with best healthcare',
              'Impact of weight on health',
              'What causes lung cancer?', 
              'Incubation period flu virus', 
              'Cost health insurance Germany',
              'Impact of personal lifestyle on health issues', 
              'How can nutrition influence mental and physical wellbeing?', 
              'Homeopathy in Europe', 
              'Mental disorder in children', 
              'medical conditions aborigines', 
              'stillbirths in the United States',
              'Is veganism healthy',
              'long-term effects of eating processed foods',
              'efficacy of quadruple bypass surgery',
              'Is Gilberts syndrome benign',
              'corruption in the pharmaceutical industry',
              'is vaping safe',
              'relationship between air pollution and respiratory health',
              'will loud music cause hearing loss']

queries = {}
c = 1
for query in query_list:
    queries[f'q{c}'] = query_prep(query)
    c += 1

#### Generate top 5 rankings per query with both OkapiBM25 and Maximum Frequency Normalization retrieval function

In [7]:
# ranking with OkapiBM25 and TF Maximum Frequency Normalization 
ranking_okapi = OkapiBM25(inv_index, queries, max_docs=5, k1=1.2, b=0.75, k3=1000)
ranking_maxfreqn = retrieve_n_rank_docs(inv_index, queries, max_docs=5)

In [8]:
queries2 = {}
c = 1
for query in query_list:
    queries2[f'q{c}'] = query, query_prep(query)
    c += 1

In [9]:
# create DataFrame for OkapiBM25
l=[]
q_id = 1
for q in ranking_okapi:  
    for r in ranking_okapi[q]:
        l.append((f'q{q_id}',queries2[q][0],queries2[q][1],r[0][0],r[0][1],r[1]))
    q_id += 1

df_okapi = pd.DataFrame(l, columns=['Query ID','Query','Stemmed Query','docID_Okapi','Result_Okapi','Score_Okapi'])

In [10]:
# create DataFrame for TF Maximum Frequency Normalization
l2 = []
q_id = 1
for q in ranking_maxfreqn:  
    for r in ranking_maxfreqn[q]:
        l2.append((f'q{q_id}',queries2[q][0],queries2[q][1],r[0][0],r[0][1],r[1]))
    q_id += 1

df_maxfreqn = pd.DataFrame(l2, columns=['Query ID','Query','Stemmed Query','docID_MaxFreq','Result_MaxFreq','Score_MaxFreq'])

In [11]:
# final DataFrame which can be used to create the Ground Truth
pd.concat([df_okapi, df_maxfreqn], axis=1)

,Query ID,Query,Stemmed Query,docID_Okapi,Result_Okapi,Score_Okapi,Query ID,Query,Stemmed Query,docID_MaxFreq,Result_MaxFreq,Score_MaxFreq
0,q1,speech language pathology,"[speech, languag, patholog]",d263,https://en.wikipedia.org/wiki/Speech_recogniti...,9.540,q1,speech language pathology,"[speech, languag, patholog]",d263,https://en.wikipedia.org/wiki/Speech_recogniti...,6.847
1,q1,speech language pathology,"[speech, languag, patholog]",d92,https://en.wikipedia.org/wiki/Artificial_intel...,8.095,q1,speech language pathology,"[speech, languag, patholog]",d92,https://en.wikipedia.org/wiki/Artificial_intel...,5.146
2,q1,speech language pathology,"[speech, languag, patholog]",d395,https://en.wikipedia.org/wiki/Barack_Obama_spe...,6.937,q1,speech language pathology,"[speech, languag, patholog]",d424,https://en.wikipedia.org/wiki/Evolutionary_psy...,5.000
3,q1,speech language pathology,"[speech, languag, patholog]",d424,https://en.wikipedia.org/wiki/Evolutionary_psy...,6.373,q1,speech language pathology,"[speech, languag, patholog]",d243,https://en.wikipedia.org/wiki/Health_informatics,4.962
4,q1,speech language pathology,"[speech, languag, patholog]",d386,https://en.wikipedia.org/wiki/Comorbidity,6.350,q1,speech language pathology,"[speech, languag, patholog]",d363,https://en.wikipedia.org/wiki/Mental_health_in...,4.928
...,...,...,...,...,...,...,...,...,...,...,...,...
145,q30,will loud music cause hearing loss,"[loud, music, caus, hear, loss]",d275,https://en.wikipedia.org/wiki/Noise_health_eff...,20.737,q30,will loud music cause hearing loss,"[loud, music, caus, hear, loss]",d275,https://en.wikipedia.org/wiki/Noise_health_eff...,10.155
146,q30,will loud music cause hearing loss,"[loud, music, caus, hear, loss]",d191,https://en.wikipedia.org/wiki/Agricultural_saf...,9.290,q30,will loud music cause hearing loss,"[loud, music, caus, hear, loss]",d191,https://en.wikipedia.org/wiki/Agricultural_saf...,7.262
147,q30,will loud music cause hearing loss,"[loud, music, caus, hear, loss]",d263,https://en.wikipedia.org/wiki/Speech_recogniti...,7.420,q30,will loud music cause hearing loss,"[loud, music, caus, hear, loss]",d263,https://en.wikipedia.org/wiki/Speech_recogniti...,6.153
148,q30,will loud music cause hearing loss,"[loud, music, caus, hear, loss]",d43,https://en.wikipedia.org/wiki/Deaf_mental_heal...,6.631,q30,will loud music cause hearing loss,"[loud, music, caus, hear, loss]",d363,https://en.wikipedia.org/wiki/Mental_health_in...,5.638
